In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
from sklearn import datasets
digits = datasets.load_digits()

In [3]:
print(len(digits.data))
print(digits.target.size)

1797
1797


In [4]:
from sklearn.datasets import fetch_openml
wine = fetch_openml(name='wine', version=1)

In [5]:
print(len(wine.data))
print(wine.target.size)

178
178


In [6]:
# 5-fold cross validation for digits dataset

digitsTrainingSetSize = int(np.ceil(0.8 * len(digits.data)))
digitsValidationSetSize = int(len(digits.data) - digitsTrainingSetSize)

xDigitsTrainingSets = []
yDigitsTrainingSets = []
xDigitsValidationSets = []
yDigitsValidationSets = []

for foldIndex in range(5):

    xValidationSet = []
    yValidationSet = []

    for index, data in enumerate(digits.data[foldIndex*digitsValidationSetSize:((foldIndex*digitsValidationSetSize)+digitsValidationSetSize)]):
        xValidationSet.append(data.tolist())
        yValidationSet.append(digits.target[index])
    
    xTrainingSet = []
    yTrainingSet = []

    for index, data in enumerate(digits.data.tolist()):
        if data not in xValidationSet:
            xTrainingSet.append(data)
            yTrainingSet.append(digits.target[index])
            
    xDigitsTrainingSets.append(xTrainingSet)
    yDigitsTrainingSets.append(yTrainingSet)
    xDigitsValidationSets.append(xValidationSet)
    yDigitsValidationSets.append(yValidationSet)
    
# 5-fold cross validation for wine dataset

wineTrainingSetSize = int(np.ceil(0.8 * len(wine.data)))
wineValidationSetSize = int(len(wine.data) - wineTrainingSetSize)

xWineTrainingSets = []
yWineTrainingSets = []
xWineValidationSets = []
yWineValidationSets = []

for foldIndex in range(5):

    xValidationSet = []
    yValidationSet = []

    for index, data in enumerate(wine.data[foldIndex*wineValidationSetSize:((foldIndex*wineValidationSetSize)+wineValidationSetSize)]):
        xValidationSet.append(data.tolist())
        yValidationSet.append(wine.target[index])
    
    xTrainingSet = []
    yTrainingSet = []

    for index, data in enumerate(wine.data.tolist()):
        if data not in xValidationSet:
            xTrainingSet.append(data)
            yTrainingSet.append(wine.target[index])
            
    xWineTrainingSets.append(xTrainingSet)
    yWineTrainingSets.append(yTrainingSet)
    xWineValidationSets.append(xValidationSet)
    yWineValidationSets.append(yValidationSet)
    

In [7]:
# one-hot encoding of y for digits dataset

numberOfDigitsTargets = 10
numberOfWineTargets = 3

for index, fold in enumerate(yDigitsTrainingSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfDigitsTargets)
        encoding[y] = 1
        encodedFold.append(encoding.tolist())
    yDigitsTrainingSets[index] = encodedFold
    
for index, fold in enumerate(yDigitsValidationSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfDigitsTargets)
        encoding[y] = 1
        encodedFold.append(encoding.tolist())
    yDigitsValidationSets[index] = encodedFold

# one-hot encoding of y for wine dataset

for index, fold in enumerate(yWineTrainingSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfWineTargets)
        encoding[int(y)-1] = 1
        encodedFold.append(encoding.tolist())
    yWineTrainingSets[index] = encodedFold
    
for index, fold in enumerate(yWineValidationSets):
    encodedFold = []
    for i, y in enumerate(fold):
        encoding = np.zeros(numberOfWineTargets)
        encoding[int(y)-1] = 1
        encodedFold.append(encoding.tolist())
    yWineValidationSets[index] = encodedFold


In [8]:
def getRandomIndices(arr, batch_size):
    indices = []
    
    if batch_size > len(arr):
        print("Error: batch size larger than size of dataset.")
        return
    
    while batch_size > 0:
        x = np.floor(np.random.random() * len(arr))
        if x not in indices:
            indices.append(int(x))
            batch_size -= 1
    
    return indices

In [9]:
# gradient descent class
 
class GradientDescent:
    
    def __init__(self, batch_size, learning_rate=.001, momentum=0.9, max_iters=10, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.batch_size = batch_size
        self.max_iters = max_iters
        self.epsilon = epsilon
            
    def run(self, gradient_fn, x, y, w):
        # TODO: grad variable and usage in loop?
        
        grad = np.inf
        t = 1
        
        while np.linalg.norm(grad) > self.epsilon and t < self.max_iters:
            # TODO: implement momentum here
            
            print("gradient descent step:", t)
            
            gradients = gradient_fn(x, y, w, self.batch_size)
            
            for c in range(len(y[0])):
                w[c] = w[c] - self.learning_rate * gradients[c]
                print("w for class: ", w[c])
            
            print("###################################################")
            
            t += 1
        
        return w

In [10]:
# logistic regression

class LogisticRegression:
    def __init__(self, add_bias=True):
        self.add_bias = add_bias
        pass
            
    def fit(self, x, y, optimizer):
        # TODO: add bias
        
        def gradient(x, y, w, batch_size):
            gradients = np.zeros(len(w)).tolist()

            indices = getRandomIndices(x, batch_size)
            print("indices for batch:", indices)

            for index in indices:
                a = np.asarray(x[index])
                b = np.asarray(y[index])
                
                print("x:", a)
                print("y:", b)
                
                # do max normalization on input for
                # numerical stability during softmax
                
                max_x = np.amax(a)
                a = a - max_x

                for c in range(len(b)):
                    w_x =  w[c] @ a
                    num = np.exp(w_x)

                    den = 0
                    for i in range(len(b)):
                        w_x =  w[i] @ a
                        den += np.exp(w_x)

                    yh_c = num/den

                    y_c = b[c]
                    cost_c = np.dot(yh_c - y_c, a)
                    
                    gradients[c] += cost_c
                    
                    print("class:", c)
                    print("softmax numerator:", num)
                    print("softmax denominator:", den)
                    print("y hat for class:", yh_c)
                    print("y actual for class:", y_c)
                    print("x gradient:", cost_c)
                    print("new gradient for class:", gradients[c])

            return gradients
        
        w0 = []
        for c in range(len(y[0])):
            w0.append(np.zeros(len(x[0])))
            
        self.w = optimizer.run(gradient, x, y, w0)
    
    def predict(self, x):
        if self.add_bias:
            x = np.column_stack([x,np.ones(N)])
        yh = x@self.w
        return yh

In [11]:
gradientDescentModel = GradientDescent(2)
logisticRegressionModel = LogisticRegression(False)


In [12]:
xtest = np.asarray([[1, 2, 3, 4], [5, 6, 7, 8], [1, 1, 2, 2]])
ytest = np.asarray([[0, 1, 0], [1, 0, 1], [0, 0, 1]])

logisticRegressionModel.fit(xWineTrainingSets[0], yWineTrainingSets[0], gradientDescentModel)

#logisticRegressionModel.fit(xtest, ytest, gradientDescentModel)



#xWineTrainingSets = []
#yWineTrainingSets = []
#xWineValidationSets = []
#yWineValidationSets = []

gradient descent step: 1
indices for batch: [42, 60]
x: [1.184e+01 2.890e+00 2.230e+00 1.800e+01 1.120e+02 1.720e+00 1.320e+00
 4.300e-01 9.500e-01 2.650e+00 9.600e-01 2.520e+00 5.000e+02]
y: [0. 1. 0.]
class: 0
softmax numerator: 1.0
softmax denominator: 3.0
y hat for class: 0.3333333333333333
y actual for class: 0.0
x gradient: [-162.72       -165.70333333 -165.92333333 -160.66666667 -129.33333333
 -166.09333333 -166.22666667 -166.52333333 -166.35       -165.78333333
 -166.34666667 -165.82666667    0.        ]
new gradient for class: [-162.72       -165.70333333 -165.92333333 -160.66666667 -129.33333333
 -166.09333333 -166.22666667 -166.52333333 -166.35       -165.78333333
 -166.34666667 -165.82666667    0.        ]
class: 1
softmax numerator: 1.0
softmax denominator: 3.0
y hat for class: 0.3333333333333333
y actual for class: 1.0
x gradient: [325.44       331.40666667 331.84666667 321.33333333 258.66666667
 332.18666667 332.45333333 333.04666667 332.7        331.56666667
 332.693333

<ipython-input-10-76caaa507a02>:37: RuntimeWarning: overflow encountered in exp
  den += np.exp(w_x)
<ipython-input-10-76caaa507a02>:32: RuntimeWarning: overflow encountered in exp
  num = np.exp(w_x)
<ipython-input-10-76caaa507a02>:39: RuntimeWarning: invalid value encountered in double_scalars
  yh_c = num/den
